## Make Database `hospital` and Tables
### also stored in `make-tables.sql`

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv('../.env')
password = os.getenv("PASS")

In [ ]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=password,
  database="hospital"
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE hospital")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""
CREATE TABLE hospital.physicians (
    ssn CHAR(11) PRIMARY KEY, 
    name VARCHAR(128), 
    primary_speciality VARCHAR(128), 
    experience_years NUMERIC(2,0) 
)""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""
CREATE TABLE hospital.patients (
    ssn CHAR(11) PRIMARY KEY, 
    name VARCHAR(128), 
    address VARCHAR(255), 
    birth_date VARCHAR(128), 
    physician_id CHAR(11), 
    FOREIGN KEY(physician_id) REFERENCES hospital.physicians(ssn) 
)""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.pharmacies (
    id NUMERIC(3,0) PRIMARY KEY, 
    name VARCHAR(128), 
    address VARCHAR(255),
    phone VARCHAR(20) 
);""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.drugs (
    id NUMERIC(3,0), 
    name VARCHAR(128) PRIMARY KEY
);""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.prescriptions (
    id NUMERIC(3,0) PRIMARY KEY, 
    patient_id CHAR(11), 
    physician_id CHAR(11), 
    drug_name VARCHAR(128), 
    date VARCHAR(128), 
    quantity NUMERIC(3,0),
    FOREIGN KEY(patient_id) REFERENCES hospital.patients(ssn), 
    FOREIGN KEY(physician_id) REFERENCES hospital.physicians(ssn),
    FOREIGN KEY(drug_name) REFERENCES hospital.drugs(name)
);
ALTER TABLE hospital.prescriptions
ADD INDEX idx_patient_drug (patient_id, drug_name);
""", multi=True)

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.adverse_interactions (
    drug_name VARCHAR(128), 
    drug_name_2 VARCHAR(128), 
    PRIMARY KEY (drug_name, drug_name_2),
    FOREIGN KEY(drug_name) REFERENCES hospital.drugs(name)
);""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.alerts (
    patient_id CHAR(11), 
    physician_id CHAR(11),
    alert_date VARCHAR(128), 
    drug1 VARCHAR(128), 
    drug2 VARCHAR(128),
    PRIMARY KEY (patient_id, physician_id, alert_date, drug1, drug2),
    FOREIGN KEY(patient_id) REFERENCES hospital.patients(ssn),
    FOREIGN KEY(physician_id) REFERENCES hospital.physicians(ssn),
    FOREIGN KEY(patient_id, drug1) REFERENCES hospital.prescriptions(patient_id, drug_name),
    FOREIGN KEY(patient_id, drug2) REFERENCES hospital.prescriptions(patient_id, drug_name)
);

ALTER TABLE hospital.alerts
ADD CONSTRAINT fk_alerts_prescriptions
FOREIGN KEY (patient_id, drug1)
REFERENCES hospital.prescriptions (patient_id, drug_name);

ALTER TABLE hospital.alerts
ADD CONSTRAINT fk_alerts_prescriptions_2
FOREIGN KEY (patient_id, drug2)
REFERENCES hospital.prescriptions (patient_id, drug_name);""", multi=True)

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.pharmacy_fills (
    prescription_id NUMERIC(3,0),
    pharmacy_id NUMERIC(3,0), 
    date VARCHAR(128), 
    cost NUMERIC(6,2),
    PRIMARY KEY(prescription_id, pharmacy_id),
    FOREIGN KEY(prescription_id) REFERENCES hospital.prescriptions(id),
    FOREIGN KEY(pharmacy_id) REFERENCES hospital.pharmacies(id)
);""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.companies (
    id NUMERIC(3,0) PRIMARY KEY, 
    name VARCHAR(128), 
    address VARCHAR(255),
    contact_phone VARCHAR(20), 
    contact_name VARCHAR(128)
);""")

In [ ]:
mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE hospital.contracts (
    id NUMERIC(3,0) PRIMARY KEY,
    company_id NUMERIC(3,0), 
    pharmacy_id NUMERIC(3,0), 
    drug_name VARCHAR(128), 
    dosage NUMERIC(5,0), 
    quantity NUMERIC(3,0), 
    date VARCHAR(128),
    price NUMERIC(6,2),
    FOREIGN KEY(company_id) REFERENCES hospital.companies(id),
    FOREIGN KEY(pharmacy_id) REFERENCES hospital.pharmacies(id),
    FOREIGN KEY(drug_name) REFERENCES hospital.drugs(name)
);""")

## Create Trigger and Stored Procedure
### run below as `procedure.sql`

```sql
USE hospital;
-- stored procedure to retrieve (speciality, experience) <--
-- given (ssn) : from physicians
DELIMITER // 
CREATE procedure get_physician_info (
    IN ssn CHAR(11),
    OUT speciality VARCHAR(128),
    OUT experience NUMERIC(2,0)
    )

BEGIN
    SELECT primary_speciality, experience_years
    INTO speciality, experience
    FROM physicians as p
    WHERE p.ssn = ssn;
END //

DELIMITER ;
```

In [ ]:
# OR do...
mycursor = mydb.cursor()
mycursor.execute("""USE hospital;
DELIMITER // 
CREATE procedure get_physician_info (
    IN ssn CHAR(11),
    OUT speciality VARCHAR(128),
    OUT experience NUMERIC(2,0)
    )

BEGIN
    SELECT primary_speciality, experience_years
    INTO speciality, experience
    FROM physicians as p
    WHERE p.ssn = ssn;
END //

DELIMITER ;""", multi=True)

### run below as `trigger.sql`

```sql
USE hospital;
-- trigger to insert a row into alerts when an update
-- to prescriptions causes an adverse reaction

DROP TRIGGER IF EXISTS drug_interaction;

DELIMITER //

CREATE TRIGGER drug_interaction AFTER INSERT ON prescriptions
FOR EACH ROW
BEGIN
    -- insert into alerts
    INSERT INTO alerts (alert_date, drug1, drug2, patient_id, physician_id)
    -- get all of the possible interactions raised
    -- using DISTINCT only for p.drug_name (to avoid duplicates when a 
    -- drug was prescribed multiple times before)
    SELECT DISTINCT NEW.date, p.drug_name, NEW.drug_name, NEW.patient_id, NEW.physician_id
    FROM   prescriptions as p, 
           adverse_interactions as ai
    WHERE  NEW.patient_id = p.patient_id
    AND    ( (ai.drug_name = p.drug_name AND ai.drug_name_2 = NEW.drug_name)
    OR       (ai.drug_name = NEW.drug_name AND ai.drug_name_2 = p.drug_name) );
END //

DELIMITER ;
```

In [ ]:
# OR do...
mycursor = mydb.cursor()
mycursor.execute("""USE hospital;

DELIMITER //

CREATE TRIGGER drug_interaction AFTER INSERT ON prescriptions
FOR EACH ROW
BEGIN
    INSERT INTO alerts (alert_date, drug1, drug2, patient_id, physician_id)
    SELECT DISTINCT NEW.date, p.drug_name, NEW.drug_name, NEW.patient_id, NEW.physician_id
    FROM   prescriptions as p, 
           adverse_interactions as ai
    WHERE  NEW.patient_id = p.patient_id
    AND    ( (ai.drug_name = p.drug_name AND ai.drug_name_2 = NEW.drug_name)
    OR       (ai.drug_name = NEW.drug_name AND ai.drug_name_2 = p.drug_name) );
END //

DELIMITER ;""")

## Insert Data into `hospital`

In [31]:
import pandas as pd

In [32]:
from dotenv import load_dotenv
import os
load_dotenv('../.env')
password = os.getenv("PASS")

In [33]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=password,
  database="hospital"
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [34]:
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('adverse_interactions',)
('alerts',)
('companies',)
('contracts',)
('drugs',)
('patients',)
('pharmacies',)
('pharmacy_fills',)
('physicians',)
('prescriptions',)


###  Run `trigger.sql` first to populate alerts

##### Physicians

In [35]:
import pandas as pd
data = pd.read_csv("../data/sql-project/physicians.csv")

mycursor = mydb.cursor()

for i,row in data.iterrows():
            sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            mydb.commit()

In [36]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM physicians LIMIT 5")
for x in mycursor:
    print(x)

('118-66-5958', 'Katz', 'Orthopedics', Decimal('3'))
('156-28-1945', 'Singh', 'Orthopedics', Decimal('25'))
('163-50-5535', 'Gold', 'Neurology', Decimal('8'))
('230-12-3219', 'El Said', 'Psychiatry', Decimal('12'))
('357-93-5814', 'Califieri', 'Cardiology', Decimal('21'))


##### Patients

In [37]:
data = pd.read_csv("../data/sql-project/patients.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO patients(ssn, name, address, birth_date, physician_id)
             VALUES (%s,%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [38]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM patients LIMIT 5")
for x in mycursor:
    print(x)

('192-33-2887', 'Jacinda Stowe', '8 Colorado Alley', '1970-04-15T00:24:26Z', '357-93-5814')
('227-08-7452', 'Bearnard Remer', '18669 Heffernan Point', '2008-01-09T05:34:30Z', '230-12-3219')
('303-13-5928', 'Krystyna Luckie', '54106 Barnett Plaza', '1950-02-11T12:20:13Z', '571-13-9020')
('360-47-2098', 'Peter Lukasen', '552 Ryan Court', '1969-01-10T19:33:03Z', '522-86-5827')
('478-34-0781', 'Florance Saiz', '7 Fair Oaks Place', '1988-11-03T23:25:38Z', '614-57-6885')


##### Pharmacies

In [39]:
data = pd.read_csv("../data/sql-project/pharmacies.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO pharmacies(id, name, address, phone)
             VALUES (%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [40]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM pharmacies LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('1'), ' Springfield Pharmacy', ' 123 Main St,  Springfield,  IL 62701', ' (217) 555-1234')
(Decimal('2'), ' Peachtree Meds', ' 456 Elm St,  Atlanta, GA 30303', ' (404) 555-5678')
(Decimal('3'), ' Lone Star Drugs', ' 789 Oak St,  Dallas, TX 75201', ' (214) 555-9101')
(Decimal('4'), ' Mile High Meds', ' 101 Pine St, Denver, CO80202', ' (303) 555-1121')
(Decimal('5'), ' Emerald City Pharmacy', ' 121 Spruce St,  Seattle,  WA 98101', ' (206) 555-1314')


##### Drugs

In [41]:
data = pd.read_csv("../data/sql-project/drugs.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO drugs(id, name)
             VALUES (%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [42]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM drugs LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('9'), 'Abnazole Toleluble')
(Decimal('3'), 'Avafoxin')
(Decimal('5'), 'Cleotrana')
(Decimal('10'), 'Dantopex Quixilinum')
(Decimal('8'), 'Divisporine Acetaclotide')


#### Adverse Interactions

In [43]:
data = pd.read_csv("../data/sql-project/adverse_reactions.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO adverse_interactions(drug_name, drug_name_2)
             VALUES (%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [44]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM adverse_interactions LIMIT 5")
for x in mycursor:
    print(x)

('Avafoxin', 'Kanulin')
('Cleotrana', 'Avafoxin')
('Cleotrana', 'Kanulin')
('Cleotrana', 'Quixiposide')
('Olanzanafine', 'Glucozepam Amcipentin')


##### Prescriptions

In [45]:
data = pd.read_csv("../data/sql-project/prescriptions.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO prescriptions(id, patient_id, physician_id, drug_name, date, quantity)
             VALUES (%s,%s,%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [46]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM prescriptions LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('1'), '478-34-0781', '614-57-6885', 'Avafoxin', '3/11/2023', Decimal('90'))
(Decimal('2'), '758-08-7274', '118-66-5958', 'Cleotrana', '3/12/2023', Decimal('10'))
(Decimal('3'), '758-08-7274', '118-66-5958', 'Primalovir', '4/11/2023', Decimal('20'))
(Decimal('4'), '758-08-7274', '118-66-5958', 'Glucozepam Amcipentin', '5/13/2023', Decimal('12'))
(Decimal('5'), '303-13-5928', '571-13-9020', 'Olanzanafine', '5/24/2023', Decimal('25'))


#### Pharmacy Fills

In [47]:
data = pd.read_csv("../data/sql-project/pharmacy_fills.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO pharmacy_fills(pharmacy_id, prescription_id, date, cost)
             VALUES (%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [48]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM pharmacy_fills LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('1'), Decimal('8'), '3/12/2023', Decimal('46.53'))
(Decimal('2'), Decimal('1'), '3/12/2023', Decimal('18.00'))
(Decimal('3'), Decimal('1'), '3/15/2023', Decimal('60.53'))
(Decimal('4'), Decimal('3'), '5/16/2023', Decimal('41.50'))
(Decimal('5'), Decimal('10'), '5/26/2023', Decimal('47.50'))


#### Companies

In [49]:
data = pd.read_csv("../data/sql-project/companies.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO companies(id, name, address, contact_name, contact_phone)
             VALUES (%s,%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [50]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM companies LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('1'), 'Goodrx', '123 Main St, San Francisco, CA', 'Holly Jolly', '123-456-7890')
(Decimal('2'), 'PHARMASEE', '456 Elm St, New York, NY', 'Faker Maker', '234-567-8901')
(Decimal('3'), 'DRUGXO', '789 Maple St, Los Angeles, CA', 'Silly Putty', '345-678-9012')
(Decimal('4'), 'Pharmachoice', '101 Pine St, Chicago, IL', 'Connie Honey', '456-789-0123')
(Decimal('5'), 'Castox', '234 Oak St, Houston, TX', 'Laxmi Kant Sheth', '567-890-1234')


#### Contracts

In [51]:
data = pd.read_csv("../data/sql-project/contracts.csv")

for i,row in data.iterrows():
    sql = """INSERT INTO contracts(id, drug_name, dosage, pharmacy_id, company_id, quantity, date, price)
             VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [52]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM contracts LIMIT 5")
for x in mycursor:
    print(x)

(Decimal('1'), Decimal('10'), Decimal('5'), 'Cleotrana', Decimal('50'), Decimal('40'), '10/3/2023', Decimal('100.00'))
(Decimal('2'), Decimal('10'), Decimal('5'), 'Primalovir', Decimal('500'), Decimal('20'), '9/26/2023', Decimal('40.50'))
(Decimal('3'), Decimal('9'), Decimal('4'), 'Kanulin', Decimal('1000'), Decimal('20'), '9/20/2023', Decimal('10.50'))
(Decimal('4'), Decimal('3'), Decimal('6'), 'Olanzanafine', Decimal('50'), Decimal('80'), '9/24/2023', Decimal('150.00'))
(Decimal('5'), Decimal('5'), Decimal('15'), 'Avafoxin', Decimal('5'), Decimal('30'), '9/29/2023', Decimal('18.00'))


#### Check Alerts

In [53]:
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM alerts")
for x in mycursor:
    print(x)

('501-47-2038', '156-28-1945', '9/22/2023', 'Avafoxin', 'Kanulin')
('501-47-2038', '156-28-1945', '9/22/2023', 'Cleotrana', 'Avafoxin')
('501-47-2038', '156-28-1945', '9/22/2023', 'Cleotrana', 'Kanulin')
('303-13-5928', '571-13-9020', '5/24/2023', 'Olanzanafine', 'Primalovir')
('303-13-5928', '571-13-9020', '5/24/2023', 'Primalovir', 'Abnazole Toleluble')
('303-13-5928', '571-13-9020', '6/22/2023', 'Olanzanafine', 'Glucozepam Amcipentin')
('478-34-0781', '614-57-6885', '9/17/2023', 'Avafoxin', 'Quixiposide')
